In [2]:
# When using Colab, make sure you run this instruction beforehand
# !pip install --upgrade cffi==1.15.0
import importlib
import cffi
# importlib.reload(cffi)
# !pip install mip

In [3]:
import mip

A company has to plan the production of 3 types of fabric, $A_1, A_2, A_3$, for a time horizon
of a single month with 22 work days. Sales forecast indicate a demand, per square meter of
fabric, of 4300, 4500, and 5225 units for, respectively, $A_1, A_2,$ and $A_3$.

The following table indicates the price of each product (Euro) and its production cost per
meter (Euro/$m^2$
). It also reports the maximum number of square meters that can be produced
in a single day (pieces/day), if the whole production capability of the factory is used to produce
only that type of fabric.

Product|$A_1$|$A_2$|$A_3$|
-------|-----|-----|-----|
Price|120|100|115|
Production cost|74|53|66|
Production amount|500|450|550|


1. Give a linear programming formulation for the problem of determining a production plan that maximizes the total revenue.


2. Which is the maximum price that the company would pay to have one extra production day per month?

3. Which is the decrease in profit due to a decrease in the demand of product $A_1$ of 10 square meters?

4. Which is the maximum amount of money that the company would invest in advertising, so as to increase the demand of $A_3$ by 100 square meters?

5. Which is the maximum decrease in the demand of $A_2$ such that the optimal solution does not change?



In [4]:
# Set definition 

# Products
J = range(3)
# Production days per month
B = 22
# Demand for product 
d = [4300, 4500, 5225]
# Price for product
r = [120, 100, 115]
# Production cost for product
c = [74, 53, 66]
# Maximum production amount
q = [500, 450, 550]

In [5]:
# Model definition
model = mip.Model()

In [6]:
# Variables definition
x = [model.add_var(lb=0) for j in J]

In [7]:
# Objective function definition
model.objective = mip.maximize(mip.xsum((r[j]-c[j])*x[j] for j in J))

In [8]:
# Demand constraints
for j in J:
    model.add_constr(x[j] <= d[j])

# Production constraints
model.add_constr(mip.xsum(x[j]/q[j] for j in J) <= B)

In [9]:
# Solve
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

### Formulation

**Sets**

*   $J$: products

**Parameters**
*   $B$: production days per month
*   $d_j$: demand for product
*   $r_j$: price for product $j \in J$
*   $c_j$: production cost for product $j \in J$
*   $q_j$: maximum production amount $j \in J$

**Variables**
*   $x_j \in \mathbb{R}^+$: square meters of product $j \in J$

**Model**

\begin{array}{lll}
\max & \sum_{j \in J} (r_j - c_j)x_j \qquad & & \text{(revenue)}\\
\textrm{s. t.}
& x_j \leq d_j & j \in J & \text{(demand)}&\\
& \sum_{j \in J} \frac{x_j}{q_j} \leq B & & \text{(production)} & \\
& x_j \in \mathbb{R}^+ & j \in J & \text{(nonnegativity)}
\end{array}

**Which is the maximum price that the company would pay to have one extra production day per month?**

This information is returned back to us with the __dual variables__ of the original problem. These give the unitary change in objective function due to a change in the right-hand side in each constraint. We are asked to assess the variation in objective function value as a function of variations in B. Consider constraint $\sum_{j \in J} \frac{x_j}{q_j}\leq B$.

In [13]:
print("Dual value:", model.constrs[2].pi)

Dual value: 10.545454545454549


In [15]:
len(model.constrs)

4

Its shadow price, given by the optimal value of the corresponding dual variable, amounts to 21150. Therefore, since for each extra production day the company will earn 21150 Euro, that is the maximum amount of money that the
company will pay to increase P. 

**Which is the decrease in profit due to a decrease in the demand of product $A_1$ of 10 square meters?**

In [ ]:
print("Dual value:", model.constrs[0].pi)

Dual value: 3.7000000000000024


The value amounts to the shadow price for the demand constraint for product $A_1$, i.e., to 3.7. By decreasing the demand by 10 square meters, the company loses 37 Euro.

**Which is the maximum amount of money that the company would invest in advertising, so as to increase the demand of $A_3$ by 100 square meters?**

In [ ]:
print("Dual value:", model.constrs[2].pi)

Dual value: 10.545454545454549


The value amounts to the shadow price for the demand constraint for product $A_3$, i.e., to 10.5455. The maximum amount of money that the company would invest to increase by 100 square meters the demand for $A_3$ amounts to 1054.55 Euro.

**Which is the maximum decrease in the demand of $A_2$ such that the optimal solution does not change?**

In [ ]:
print("Dual value:", model.constrs[1].pi)
print(model.vars[1].x)

Dual value: -0.0
1755.000000000001


The shadow price for the demand constraint for product A2 is 0. Any variation in the demand of such product will not change the value of the objective function. The optimality
of the current basic solution is maintained for variations within $[1755, ∞)$.